# Crawler to retrieve houses from Funda.nl

<h1 align='center'>=========================================================</h1>

# INTEGRATION WITH GOOGLE DRIVE SPREADSHEET (into df_current)

In [12]:
google_json_key = 'buying-house-283614-38d7fde831c8.json'
filename = "buy house notes"
worksheet_name = "houses"
worksheet_name = "houses_utrecht"

In [13]:
import gspread 
from gspread_dataframe import get_as_dataframe, set_with_dataframe 
from oauth2client.service_account import ServiceAccountCredentials

scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_json_key, scope) 
gc = gspread.authorize(credentials)

worksheet = gc.open(filename).worksheet(worksheet_name)
df_current = get_as_dataframe(worksheet,index_column=0)

<h1 align='center'>=========================================================</h1>

# PART II: defining a function to automatic crawler all houses to buy from funda.nl! retrieve only houses that are not in a oldcsv converted in a df

Crawl only URLs that are not in the CSV

In [29]:
import pandas as pd 

import cfscrape
from lxml import etree
header = {'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9',
              'Accept-Encoding': 'gzip, deflate, sdch',
              'Accept-Language' : 'en-US,en;q=0.9,nl;q=0.8',
              'Cache-Control' : 'max-age=0',
              'Connection': 'keep-alive',
              'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.116 Safari/537.36'}

scraper = cfscrape.create_scraper()


def funda_automatic_crawler (city, price_max, df_oldfile):
    
    page = 1
    links = []
    
    while True:
        url="https://www.funda.nl/koop/"+str(city)+"/0-"+str(price_max)+"/woonhuis/p"+str(page)
#         print(url)
        scraped_html=scraper.get(url,headers=header).content
#         print(scraped_html.decode("utf-8"))
        html = etree.HTML(scraped_html)
#         print(html)
        
        df_links = pd.DataFrame(html.xpath("//a/@href"))
        
        if len(df_links)==0:
            print("Funda may BLOCKED US!")
            break

        df_links_unique ="https://www.funda.nl"+df_links[df_links[0].str.contains("/koop/"+str(city)+"/huis-")].drop_duplicates()
#         print(df_links_unique)

        links.extend(df_links_unique[0].tolist())
        
        print("Links from page",page,"retrieved!")
        
        total_entries = int(html.xpath("//button[@class='mobile-search-sticky-button fd-btn fd-btn--primary']/span/text()")[0].replace("koopwoningen",'').strip())
#         print(total_entries)
        
        if page * 10 > total_entries:
            break
        else:
            page = page + 1
#   -------------------------------------------
#   -------------------------------------------
    links_unique = list(set(links))
    df = pd.DataFrame()
    for url in links_unique:
        if len(df_oldfile[df_oldfile['link'].str.contains(url)])>0:
            print(url,":alredy in the oldcsv")
            continue    
        
        scraped_html=scraper.get(url,headers=header).content.decode("utf-8") 
#         print(scraped_html.decode("utf-8"))
        html = etree.HTML(scraped_html)             
        
        try:
            address = html.xpath("//title/text()")[0].replace('Huis te koop: ','').replace('[funda]','')
        except:
            address = ""
            
        try:
            price = html.xpath("//strong[@class='object-header__price']/text()")[0].replace('€ ','').replace(' /mnd','').replace('.','').replace(' kk','').replace('von', '').replace(' ','')
        except:
            price = ""
            
        try:
            bouwjaar = html.xpath("//dl[@class='object-kenmerken-list']")[1].xpath("dd/text()")[2].replace('\r\n','').replace(' ','')
        except:
            bouwjaar =""
            
        try:
            kamers = html.xpath("//dl[@class='object-kenmerken-list']")[3].xpath("dd/text()")[0].replace('\r\n','')
        except:
            kamers = ""
        
        try:
            woonen_m2 = html.xpath("//dd[@class='object-kenmerken-group-list']/dl/dd[1]/text()")[0].replace('\r\n','').replace(' m²','').replace('.0','').replace('.','')
        except:
            woonen_m2 = ""
            
        try:
            perceel_m2 = html.xpath("//dd[@class='object-kenmerken-group-list']/dl/dd[1]/text()")[1].replace('\r\n','').replace(' m²','').replace('.0','').replace('.','').replace(' (deelperceel)','')
        except:
            perceel_m2 =""
        
        try:
            makelaars = html.xpath("//a[@class='object-contact-aanbieder-link']/text()")[0]
        except:
            makelaars = ""
        
        try:
            makelaars_phone = html.xpath("//span[@class='fd-completely-hidden fd-display-inline-block--bp-m']/text()")[0].replace('\r\n','').replace('-','').replace('(+31) ','').replace(' ','')
        except:
            makelaars_phone = ""
            
        try:
            energy = html.xpath("//span[contains(@class, 'energielabel')]/text()")[0].replace('\r','').replace('\n','').replace(' ','')
        except:
            energy = ""
        
        print("Retrieved:", address)
        df_temp = pd.DataFrame({'link':url,
                                'address':[address],
                                'city':[city],
                                'price':[price],
                                'bouwjaar': [bouwjaar],
                                'kamers': [kamers],
                                'woonen_m2':[woonen_m2],
                                'perceel_m2':[perceel_m2],
                                'makelaars': [makelaars],
                                'makelaars_phone':[makelaars_phone],
                               'energy':[energy]})
        
        df = pd.concat([df,df_temp], axis=0, ignore_index=True)
    
    try:
        df['slaapkamers'] = df['kamers'].apply(lambda x: str(x.split(' slaapkamers')[0])[-1] if 'slaapkamers' in str(x)  else str(x).replace(' kamer','').replace('s',''))
    except:
        df['slaapkamers'] = ""

    try:
        df['bouwjaar'] = df['bouwjaar'].apply(lambda x: x.split('-')[0])
    except:
        df['slaapkamers'] = ""
    return df

<h1 align='center'>=========================================================</h1>

## Using the function

**Note:** we use the df_current (coming from Google Drive Spreadsheet)

In [30]:
df_newentries = funda_automatic_crawler('utrecht',350000,df_current)

Links from page 1 retrieved!
Links from page 2 retrieved!
Links from page 3 retrieved!
Links from page 4 retrieved!
Links from page 5 retrieved!
Links from page 6 retrieved!
Links from page 7 retrieved!
Retrieved: Torontodreef 24 3564 KR Utrecht 
Retrieved: Houstondreef 15 3564 KL Utrecht 
Retrieved: Patrimoniumstraat 45 3555 GL Utrecht 
Retrieved: Priokstraat 25 3531 XM Utrecht 
Retrieved: Bremstraat 18 3551 TD Utrecht 
Retrieved: Nicolaasweg 32 3581 VJ Utrecht 
Retrieved: Fregatstraat 9 3534 RA Utrecht 
Retrieved: De Bazelstraat 11 3555 CP Utrecht 
Retrieved: Asterstraat 22 3551 SX Utrecht 
Retrieved: Abel Tasmanstraat 76 3531 GW Utrecht 
Retrieved: Stanleylaan 3 3526 TA Utrecht 
Retrieved: Rhônedreef 19 3561 VA Utrecht 
Retrieved: Elzas 12 3524 RW Utrecht 
Retrieved: Van Meursstraat 69 3532 CH Utrecht 
Retrieved: Makassarstraat 89 3531 VM Utrecht 
Retrieved: Van Zijlstraat 1 3552 XR Utrecht 
Retrieved: Adelboldstraat 14 3553 SJ Utrecht 
Retrieved: Gerard Revestraat 23 3544 KK Utrech

In [31]:
df_newentries['id'] = ""
df_newentries['bike_time'] = ""
df_newentries['status'] = "new"
df_newentries['wos_waarde'] = ""
df_newentries['huispedia'] = ""

<h1 align='center'>=========================================================</h1> 

# PART III: enriching the data

# Bike Distance from/to SOMEWHERE

In [32]:
%run gmaps_bike_distance_duration.ipynb

Northwave Group HQ, Van Deventerlaan, Utrecht -> maxima park : 22 mins


In [33]:
# point_b = 'Enschede Railway Station, Enschede'
point_b = 'Van Deventerlaan 31, 3528 AG Utrecht'
 
df_newentries['bike_time'] = df_newentries['address'].apply(lambda x: gmaps_bike_time(gmaps_key,x,point_b))

Torontodreef 24 3564 KR Utrecht  -> Van Deventerlaan 31, 3528 AG Utrecht : 29 mins
Houstondreef 15 3564 KL Utrecht  -> Van Deventerlaan 31, 3528 AG Utrecht : 29 mins
Patrimoniumstraat 45 3555 GL Utrecht  -> Van Deventerlaan 31, 3528 AG Utrecht : 25 mins
Priokstraat 25 3531 XM Utrecht  -> Van Deventerlaan 31, 3528 AG Utrecht : 14 mins
Bremstraat 18 3551 TD Utrecht  -> Van Deventerlaan 31, 3528 AG Utrecht : 20 mins
Nicolaasweg 32 3581 VJ Utrecht  -> Van Deventerlaan 31, 3528 AG Utrecht : 22 mins
Fregatstraat 9 3534 RA Utrecht  -> Van Deventerlaan 31, 3528 AG Utrecht : 16 mins
De Bazelstraat 11 3555 CP Utrecht  -> Van Deventerlaan 31, 3528 AG Utrecht : 23 mins
Asterstraat 22 3551 SX Utrecht  -> Van Deventerlaan 31, 3528 AG Utrecht : 21 mins
Abel Tasmanstraat 76 3531 GW Utrecht  -> Van Deventerlaan 31, 3528 AG Utrecht : 14 mins
Stanleylaan 3 3526 TA Utrecht  -> Van Deventerlaan 31, 3528 AG Utrecht : 7 mins
Rhônedreef 19 3561 VA Utrecht  -> Van Deventerlaan 31, 3528 AG Utrecht : 27 mins
Elz

<h1 align='center'>=========================================================</h1> 

## Merging the new entries with a Google drive workbook

In [34]:
df_merged = df_current.append(df_newentries).reset_index(drop=True)

/usr/local/lib/python3.7/site-packages/pandas/core/frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


## Saving merged df into a Google drive workbook

In [35]:
set_with_dataframe(worksheet, df_merged[['id','status','link','address','bike_time','makelaars','makelaars_phone','price','huispedia','wos_waarde','bouwjaar','perceel_m2','woonen_m2','slaapkamers', 'energy']])